In [3]:
import torch.nn as nn
import torch
import numpy as np

In [4]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [5]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [6]:
targets 

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

###Dataset and DataLoader

In [7]:
from torch.utils.data import TensorDataset

In [8]:
# define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [9]:
from torch.utils.data import DataLoader


In [10]:
# Define data Loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [11]:
for xb, yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[ 69.,  96.,  70.],
        [ 68.,  97.,  70.],
        [ 88., 134.,  59.],
        [ 68.,  96.,  71.],
        [101.,  44.,  37.]])
tensor([[103., 119.],
        [102., 120.],
        [118., 132.],
        [104., 118.],
        [ 21.,  38.]])


###nn.Linear

In [12]:
# Define Model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3527,  0.4234,  0.2645],
        [ 0.3583,  0.5712, -0.0950]], requires_grad=True)
Parameter containing:
tensor([-0.4586, -0.1079], requires_grad=True)


In [13]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.3527,  0.4234,  0.2645],
         [ 0.3583,  0.5712, -0.0950]], requires_grad=True),
 Parameter containing:
 tensor([-0.4586, -0.1079], requires_grad=True)]

In [14]:
preds = model(inputs)
preds

tensor([[ 13.5331,  60.2313],
        [ 21.6296,  76.6806],
        [ 40.9296, 102.0922],
        [ -8.4443,  57.4824],
        [ 34.3636,  72.7984],
        [ 12.7570,  60.0184],
        [ 21.4707,  76.0145],
        [ 40.8414, 102.3555],
        [ -7.6682,  57.6953],
        [ 34.9808,  72.3451],
        [ 13.3742,  59.5652],
        [ 20.8535,  76.4677],
        [ 41.0885, 102.7584],
        [ -9.0615,  57.9357],
        [ 35.1397,  73.0113]], grad_fn=<AddmmBackward>)

###Loss Function


In [15]:
# Loss Function
import torch.nn.functional as F


In [16]:
# Define Loss Function
loss_fn = F.mse_loss

In [17]:
loss = loss_fn(model(inputs), targets)
loss

tensor(2112.7656, grad_fn=<MseLossBackward>)

### Optimisers

In [18]:
# Define Optimiser
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

### Train Model

In [22]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [23]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 75.8673
Epoch [20/100], Loss: 135.5262
Epoch [30/100], Loss: 163.4100
Epoch [40/100], Loss: 124.4512
Epoch [50/100], Loss: 65.1787
Epoch [60/100], Loss: 34.3294
Epoch [70/100], Loss: 44.4280
Epoch [80/100], Loss: 31.7434
Epoch [90/100], Loss: 56.8022
Epoch [100/100], Loss: 51.0257


In [24]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.4742,  71.9637],
        [ 81.1700,  96.6553],
        [119.7314, 140.5975],
        [ 23.8490,  45.2940],
        [ 98.4708, 107.2159],
        [ 56.2722,  70.9249],
        [ 80.8177,  95.9158],
        [119.9347, 140.7832],
        [ 25.0511,  46.3328],
        [ 99.3206, 107.5153],
        [ 57.1219,  71.2242],
        [ 79.9679,  95.6165],
        [120.0837, 141.3370],
        [ 22.9993,  44.9946],
        [ 99.6729, 108.2547]], grad_fn=<AddmmBackward>)

In [25]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [26]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.8389, 68.1647]], grad_fn=<AddmmBackward>)